# <center>Notebook para construção da tabela de países<center/>

Bases de Dados utilizadas:
* DBpedia
* WHO GHO (Athena API)

Bibliotecas

In [1]:
import io
import requests
import json
import pandas as pd
import xml.dom.minidom as mdom

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### DBPedia (GINI, IDH e PIB per capita) e WHO GHO (nome, sigla, região, classficação do World Bank)

In [2]:
# It returns a list of N countries in dimension N x  
# GDB PPP per capita, Gini, HDI
def getDataByCountry(countriesList):
    out = []
    notFound = []
    missing_info = {}
    requested_info = {'gini': 'http://dbpedia.org/property/gini', 
                     'gdpPppPerCapita': 'http://dbpedia.org/property/gdpPppPerCapita',
                     'hdi': 'http://dbpedia.org/property/hdi', 
                     'populationTotal': 'http://dbpedia.org/ontology/populationTotal'}
    for i in countriesList:
        info = []
        try:
            url = 'http://dbpedia.org/data/' + str(i['country']) +  '.jsod'
            js = requests.get(url).json()
            for ri in requested_info:
                if requested_info[ri] in js['d']['results'][0]:
                    info.append(js['d']['results'][0][requested_info[ri]])
                else:
                    info.append('')
                    if i['country'] in missing_info:
                        missing_info[i['country']].append(ri)
                    else:
                         missing_info[i['country']] = [ri]
            out.append({'country': i['country'].replace('_', ' '),
                  'label': i['label'],
                  'region': i['region'],
                  'wb_class': i['wb_class'],
                  'gdbPPPperCapita': info[1],
                  'gini': round(float(info[0])*10) if info[0] != '' else info[0],
                  'hdi': round(float(info[2])*1000) if info[2] != '' else info[2],
                  'populationTotal': info[3]})
        except:
            print('We were not able to get %s!' %(i['country']))
            notFound.append(i)
    return out, notFound, missing_info

In [3]:
#Return the index of an object in an array based on a field value
def obj_index(array, field_name, value):
    for i in range(len(array)):
        if array[i][field_name] == value:
            return i
    return -1

In [4]:
#XML search in DBpedia XML format to retrive missing information in the json format
def XMLsearchData(partiallyFound, notFound, missingInfo):
    count = 0
    requested_info = {'gini': 'dbp:gini', 
                     'gdpPppPerCapita': 'dbp:gdpPppPerCapita',
                     'hdi': 'dbp:hdi', 
                     'populationTotal': 'dbo:populationTotal'}
    for country in notFound:
        url = 'http://dbpedia.org/data/' + str(country['country']) + '.atom'
        try:
            xml = mdom.parseString(requests.get(url).content)
            partiallyFound.append({
                      'country': country['country'].replace('_', ' '),
                      'label': country['label'],
                      'region': country['region'],
                      'wb_class': country['wb_class'],
                      'gini': round(float(xml.getElementsByTagName('dbp:gini')[0].firstChild.nodeValue)*10) if len(xml.getElementsByTagName('dbp:gini')) > 0 else '',
                      'gdbPPPperCapita': round(float(xml.getElementsByTagName('dbp:gdpPppPerCapita')[0].firstChild.nodeValue)) if len(xml.getElementsByTagName('dbp:gdpPppPerCapita')) > 0 else '',
                      'hdi': round(float(xml.getElementsByTagName('dbp:hdi')[0].firstChild.nodeValue)*1000) if len(xml.getElementsByTagName('dbp:hdi')) > 0 else '',
                      'populationTotal': round(float(xml.getElementsByTagName('dbo:populationTotal')[0].firstChild.nodeValue)) if len(xml.getElementsByTagName('dbo:populationTotal')) > 0 else ''
                      })
        except:
            count+=1
            partiallyFound.append({
                  'country': country['country'].replace('_', ' '),
                  'label': country['label'],
                  'region': country['region'],
                  'wb_class': country['wb_class'],
                  'gini': '',
                  'gdbPPPperCapita': '',
                  'hdi': '',
                  'populationTotal': ''
                  })
            print(country['country'] + ' not Found')
    for country in missingInfo:
        url = 'http://dbpedia.org/data/' + str(country) + '.atom'
        try:
            print('Trying to get missing information ({missing}) for {country}'.format(missing=missingInfo[country], country=country))
            retrieved = 0
            xml = mdom.parseString(requests.get(url).content)
            index = obj_index(partiallyFound, 'country', country.replace('_', ' '))
            for info in missingInfo[country]:
                if len(xml.getElementsByTagName(requested_info[info])) > 0:
                    retrieved += 1
                    if info == 'gini':
                        partiallyFound[index]['gini'] = round(float(xml.getElementsByTagName('dbp:gini')[0].firstChild.nodeValue)*10)
                    elif info == 'hdi':
                        partiallyFound[index]['hdi'] = round(float(xml.getElementsByTagName('dbp:hdi')[0].firstChild.nodeValue)*1000)
                    else:
                        partiallyFound[index][info] = round(float(xml.getElementsByTagName(missingInfo[info])[0].firstChild.nodeValue))
            print("{}/{} filds were retrieved".format(retrieved, len(missingInfo[country])))
        except:
            continue
    return partiallyFound, count

In [5]:
# Get list of countries used in WHO dataset
def getCountriesList():
    countriesList = []
    url = 'https://apps.who.int/gho/athena/api/GHO/HIV_0000000026?format=json'
    js = requests.get(url).json()
    for i in js['dimension'][4]['code']:  
        newCountry = i['display'].split('(')[0].replace(' ', '_')
        if newCountry[len(newCountry) - 1] == '_': 
            countriesList.append({'country': newCountry[:-1], 'label': i['label'], 'region': i['attr'][obj_index(i['attr'], 'category', 'WHO_REGION')]['value'], 'wb_class': i['attr'][obj_index(i['attr'], 'category', 'WORLD_BANK_INCOME_GROUP')]['value']})
        else:
            countriesList.append({'country': newCountry, 'label': i['label'], 'region': i['attr'][obj_index(i['attr'], 'category', 'WHO_REGION')]['value'], 'wb_class': i['attr'][obj_index(i['attr'], 'category', 'WORLD_BANK_INCOME_GROUP')]['value']})
    return countriesList

In [6]:
countriesList = getCountriesList();
info, notFound, missingInfo = getDataByCountry(countriesList);

We were not able to get Afghanistan!
We were not able to get Albania!
We were not able to get Belgium!
We were not able to get Canada!
We were not able to get Central_African_Republic!
We were not able to get Djibouti!
We were not able to get Ecuador!
We were not able to get Eritrea!
We were not able to get Ethiopia!
We were not able to get Fiji!
We were not able to get Guinea-Bissau!
We were not able to get Guyana!
We were not able to get Israel!
We were not able to get Japan!
We were not able to get Lesotho!
We were not able to get Malawi!
We were not able to get Malaysia!
We were not able to get Mongolia!
We were not able to get Spain!
We were not able to get Sudan!
We were not able to get Eswatini!
We were not able to get Uzbekistan!


In [7]:
print("Países encontrados: {}".format(len(info)))
print("Países encontrados com informação faltando: {}".format(len(missingInfo)))
print("Países não encontrados: {}".format(len(notFound)))

Países encontrados: 148
Países encontrados com informação faltando: 56
Países não encontrados: 22


In [8]:
xmlResul, noInfo = XMLsearchData(info, notFound, missingInfo)

Afghanistan not Found
Belgium not Found
Canada not Found
Ethiopia not Found
Guyana not Found
Japan not Found
Malawi not Found
Uzbekistan not Found
Trying to get missing information (['populationTotal']) for Angola
0/1 filds were retrieved
Trying to get missing information (['populationTotal']) for Australia
0/1 filds were retrieved
Trying to get missing information (['gini', 'gdpPppPerCapita', 'hdi', 'populationTotal']) for Bahamas
0/4 filds were retrieved
Trying to get missing information (['gini']) for Bahrain
0/1 filds were retrieved
Trying to get missing information (['gini', 'populationTotal']) for Barbados
0/2 filds were retrieved
Trying to get missing information (['gini']) for Belize
0/1 filds were retrieved
Trying to get missing information (['populationTotal']) for Bosnia_and_Herzegovina
0/1 filds were retrieved
Trying to get missing information (['gini', 'gdpPppPerCapita', 'hdi', 'populationTotal']) for Botswana
0/4 filds were retrieved
Trying to get missing information (['g

In [9]:
print('Total de países registrados: {}'.format(len(xmlResul)))
print('Países sem nenhuma informação: {}'.format(noInfo))

Total de países registrados: 170
Países sem nenhuma informação: 8


In [10]:
print('For a table created in the following order: country, gdb PPP per Capita, gini, hdi')
for i in info:
    country = i['country'].replace('_', ' ')
    print("""\
    INSERT INTO Pais VALUES('%s', %s, %s, %s);\
    """%(country, i['gdbPPPperCapita'], i['gini'], i['hdi']))
for i in xmlResul:
    country = i['country'].replace('_', ' ')
    print("""\
    INSERT INTO Pais VALUES('%s', %s, %s, %s);\
    """%(country, i['gdbPPPperCapita'], i['gini'], i['hdi']))

For a table created in the following order: country, gdb PPP per Capita, gini, hdi
    INSERT INTO Pais VALUES('Algeria', 14610.0, 353, 736);    
    INSERT INTO Pais VALUES('Angola', 7501.0, 427, 532);    
    INSERT INTO Pais VALUES('Argentina', 22554.0, 423, 836);    
    INSERT INTO Pais VALUES('Armenia', 8468.0, 315, 733);    
    INSERT INTO Pais VALUES('Australia', 47318.0, 336, 935);    
    INSERT INTO Pais VALUES('Austria', 48098.0, 276, 885);    
    INSERT INTO Pais VALUES('Azerbaijan', 17500.0, 337, 751);    
    INSERT INTO Pais VALUES('Bahamas', , , );    
    INSERT INTO Pais VALUES('Bahrain', 29789.0, , 824);    
    INSERT INTO Pais VALUES('Bangladesh', 3581.0, 321, 570);    
    INSERT INTO Pais VALUES('Barbados', 16653.0, , 785);    
    INSERT INTO Pais VALUES('Belarus', 17440.0, 265, 798);    
    INSERT INTO Pais VALUES('Belize', 8495.0, , 715);    
    INSERT INTO Pais VALUES('Benin', 2025.0, 365, 480);    
    INSERT INTO Pais VALUES('Bhutan', 8196.0, 387, 605)

## Exportação para CSV

In [11]:
df = pd.DataFrame(xmlResul,columns=['country', 'label', 'region', 'wb_class', 'gini', 'gdbPPPperCapita', 'hdi', 'populationTotal'])

In [12]:
df.to_csv('../data/Filtered Data/countries.csv', index=False)